<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [5]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [8]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

('Numpy: ', '1.16.5')
('Pandas: ', u'0.24.2')


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [12]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...
()
  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     
()
  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                    

C:\Users\Perrine Mignot\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
C:\Users\Perrine Mignot\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  import sys
C:\Users\Perrine Mignot\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  if sys.path[0] == '':


?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [72]:
#ANSWER:
bikes = pd.read_table('bikeshare.csv', header = 0, sep = ',')
print(bikes.head())

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


C:\Users\Perrine Mignot\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [32]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [36]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
5,1,0.4,1.7,3.9,5.4,Setosa
6,1,0.3,1.4,3.4,4.6,Setosa
7,1,0.2,1.5,3.4,5.0,Setosa
8,1,0.2,1.4,2.9,4.4,Setosa
9,1,0.1,1.5,3.1,4.9,Setosa


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [56]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Data', header = 0, nrows = 20, index_col = 'Species_No', usecols = [0,1,2,3,4,5])
df

,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
Species_No,,,,,
1,0.2,1.4,3.5,5.1,Setosa
1,0.2,1.4,3.0,4.9,Setosa
1,0.2,1.3,3.2,4.7,Setosa
1,0.2,1.5,3.1,4.6,Setosa
1,0.2,1.4,3.6,5.0,Setosa
1,0.4,1.7,3.9,5.4,Setosa
1,0.3,1.4,3.4,4.6,Setosa
1,0.2,1.5,3.4,5.0,Setosa
1,0.2,1.4,2.9,4.4,Setosa


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [54]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-07-05T09:39:47.375Z,9.8744,-84.3523,10,4.30,mb,NaN,154,0.269,1.13,...,2020-07-05T10:13:29.176Z,"5 km NW of Santiago, Costa Rica",earthquake,6.30,2.00,0.075,50,reviewed,us,us
1,2020-07-05T09:21:27.320Z,17.9395,-66.9575,7,2.98,md,23.0,202,0.086,0.13,...,2020-07-05T09:57:03.191Z,"6 km SW of Guánica, Puerto Rico",earthquake,0.42,0.23,0.080,5,reviewed,pr,pr


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [62]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                                             Bank Name                City  \
 0                                 The First State Bank       Barboursville   
 1                                   Ericson State Bank             Ericson   
 2                     City National Bank of New Jersey              Newark   
 3                                        Resolute Bank              Maumee   
 4                                Louisa Community Bank              Louisa   
 5                                 The Enloe State Bank              Cooper   
 6                  Washington Federal Bank for Savings             Chicago   
 7      The Farmers and Merchants State Bank of Argonia             Argonia   
 8                                  Fayette County Bank          Saint Elmo   
 9    Guaranty Bank, (d/b/a BestBank in Georgia & Mi...           Milwaukee   
 10                                      First NBC Bank         New Orleans   
 11                                       Proficio B

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [63]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,category,datetime,integer
0,0,2015-01-01 00:00:00,5
1,0,2015-01-01 00:00:01,5
10,0,2015-01-01 00:00:10,5
11,0,2015-01-01 00:00:11,5
12,0,2015-01-01 00:00:12,8


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [97]:
#ANSWER
bikes.shape
#Answer: 17 columns and 17379 rows

(17379, 17)

What are the column names in `bikes`?

In [68]:
#ANSWER
bikes.columns
#Answer: u'instant', u'dteday', u'season', u'yr', u'mnth', u'hr', u'holiday', u'weekday', u'workingday', u'weathersit', u'temp', u'atemp', u'hum',u'windspeed', u'casual', u'registered', u'cnt'

Index([u'instant', u'dteday', u'season', u'yr', u'mnth', u'hr', u'holiday',
       u'weekday', u'workingday', u'weathersit', u'temp', u'atemp', u'hum',
       u'windspeed', u'casual', u'registered', u'cnt'],
      dtype='object')

What are the data types of these columns?

In [71]:
#ANSWER
bikes.dtypes
#12 columns are integers (full numbers) except for dteday which is a date and the following which are floats (numbers with decimals): temp, atemp, hum, windespeed

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [98]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

In [78]:
#ANSWER bikes[['season']] returns a cleaner table with headings and grid whereas bikes['season'] has row data with no structure
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


How would we use object notation to show the first 4 rows of `atemp`?

In [95]:
#ANSWER
bikes.loc[0:4,['atemp']]

,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879
4,0.2879


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [102]:
#E: bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
bikes[[0,0]]

KeyError: u"None of [Int64Index([0, 0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [105]:
#ANSWER
bikes[0:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [107]:
#ANSWER
bikes.iloc[:,1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
5        2011-01-01
6        2011-01-01
7        2011-01-01
8        2011-01-01
9        2011-01-01
10       2011-01-01
11       2011-01-01
12       2011-01-01
13       2011-01-01
14       2011-01-01
15       2011-01-01
16       2011-01-01
17       2011-01-01
18       2011-01-01
19       2011-01-01
20       2011-01-01
21       2011-01-01
22       2011-01-01
23       2011-01-01
24       2011-01-02
25       2011-01-02
26       2011-01-02
27       2011-01-02
28       2011-01-02
29       2011-01-02
            ...    
17349    2012-12-30
17350    2012-12-30
17351    2012-12-30
17352    2012-12-30
17353    2012-12-30
17354    2012-12-30
17355    2012-12-31
17356    2012-12-31
17357    2012-12-31
17358    2012-12-31
17359    2012-12-31
17360    2012-12-31
17361    2012-12-31
17362    2012-12-31
17363    2012-12-31
17364    2012-12-31
17365    2012-12-31
17366    2012-12-31
17367    2012-12-31


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [108]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [127]:
#ANSWER
bikes.isnull().sum().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [112]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [135]:
#ANSWER: acronym for Not a Number. 
bikes.fillna(0, inplace = True) 
print(bikes)

       instant      dteday  season  yr  mnth  hr  holiday  weekday  \
0            1  2011-01-01       1   0     1   0        0        6   
1            2  2011-01-01       1   0     1   1        0        6   
2            3  2011-01-01       1   0     1   2        0        6   
3            4  2011-01-01       1   0     1   3        0        6   
4            5  2011-01-01       1   0     1   4        0        6   
5            6  2011-01-01       1   0     1   5        0        6   
6            7  2011-01-01       1   0     1   6        0        6   
7            8  2011-01-01       1   0     1   7        0        6   
8            9  2011-01-01       1   0     1   8        0        6   
9           10  2011-01-01       1   0     1   9        0        6   
10          11  2011-01-01       1   0     1  10        0        6   
11          12  2011-01-01       1   0     1  11        0        6   
12          13  2011-01-01       1   0     1  12        0        6   
13          14  2011

Apply the Pandas `isna` function to the following data objects:

In [136]:
x = 2.3
y = np.nan
print(x, y)

(2.3, nan)


In [141]:
#ANSWER
print(pd.isna(x), pd.isna(y))

(False, True)


In [137]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [0]:
#ANSWER


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [143]:
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [144]:
#ANSWER
print('instant:', bikes['instant'].min(), 'to', bikes['instant'].max())
print('dteday:', bikes['dteday'].min(), 'to', bikes['dteday'].max())
print('windspeed:', bikes['windspeed'].min(), 'to', bikes['windspeed'].max())

('instant:', 1, 'to', 17379)
('dteday:', '2011-01-01', 'to', '2012-12-31')
('windspeed:', 0.0, 'to', 0.8507)


Compute and print the overall minimum and maximum of the numeric data columns:

In [145]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

(0, '2012-12-31')

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [146]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [148]:
#ANSWER
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [149]:
type(bikes['atemp'])

pandas.core.series.Series

In [3]:
bikes.sample(5)

NameError: name 'bikes' is not defined

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [154]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)     
print(atemp_level)

0           (0.25, 0.5]
1           (0.25, 0.5]
2           (0.25, 0.5]
3           (0.25, 0.5]
4           (0.25, 0.5]
5           (0.25, 0.5]
6           (0.25, 0.5]
7           (0.25, 0.5]
8           (0.25, 0.5]
9           (0.25, 0.5]
10          (0.25, 0.5]
11          (0.25, 0.5]
12          (0.25, 0.5]
13          (0.25, 0.5]
14          (0.25, 0.5]
15          (0.25, 0.5]
16          (0.25, 0.5]
17          (0.25, 0.5]
18          (0.25, 0.5]
19          (0.25, 0.5]
20          (0.25, 0.5]
21          (0.25, 0.5]
22          (0.25, 0.5]
23          (0.25, 0.5]
24          (0.25, 0.5]
25          (0.25, 0.5]
26          (0.25, 0.5]
27          (0.25, 0.5]
28          (0.25, 0.5]
29          (0.25, 0.5]
              ...      
17349    (-0.001, 0.25]
17350       (0.25, 0.5]
17351    (-0.001, 0.25]
17352    (-0.001, 0.25]
17353    (-0.001, 0.25]
17354    (-0.001, 0.25]
17355    (-0.001, 0.25]
17356    (-0.001, 0.25]
17357    (-0.001, 0.25]
17358    (-0.001, 0.25]
17359    (-0.001

What is `atemp_level`?

In [0]:
#ANSWER: it is a split of the atemp column into 4 bins between 0 and 1


Here is a random sample of `atemp_level`:

In [155]:
atemp_level.sample(5)          

11920       (0.5, 0.75]
17250    (-0.001, 0.25]
14082       (0.5, 0.75]
14880       (0.25, 0.5]
14689       (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [156]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)  

426     mild
9244    mild
2309    mild
2623    mild
1445    mild
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [158]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes[(bikes.atemp_level == 'mild') & (bikes.season == 2)].season.count()

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [170]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [160]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [171]:
rock = pd.read_csv("rock.csv")
rock.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Check Column Names

Check column names and clean.

In [162]:
rock.columns

Index([u'Song Clean', u'ARTIST CLEAN', u'Release Year', u'COMBINED', u'First?',
       u'Year?', u'PlayCount', u'F*G'],
      dtype='object')

In [177]:
rock.columns = rock.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('?', '').str.replace('*', '').str.replace('(', '').str.replace(')', '')
rock.columns

Index([u'song_clean', u'artist_clean', u'release_year', u'combined', u'first',
       u'year', u'playcount', u'fg'],
      dtype='object')

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [165]:
rock.isnull().sum()

Song Clean        0
ARTIST CLEAN      0
Release Year    577
COMBINED          0
First?            0
Year?             0
PlayCount         0
F*G               0
dtype: int64

In [195]:
rock.fillna(0, inplace = True) 
print(rock)

                                       song_clean   artist_clean  \
0                                Caught Up in You    .38 Special   
1                                    Fantasy Girl    .38 Special   
2                                 Hold On Loosely    .38 Special   
3                          Rockin' Into the Night    .38 Special   
4                               Art For Arts Sake           10cc   
5                                      Kryptonite   3 Doors Down   
6                                           Loser   3 Doors Down   
7                                   When I'm Gone   3 Doors Down   
8                                      What's Up?  4 Non Blondes   
9                                      Take On Me           a-ha   
10                          Baby, Please Don't Go          AC/DC   
11                                  Back In Black          AC/DC   
12                                        Big Gun          AC/DC   
13                         CAN'T STOP ROCK'N'ROL

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [196]:
rock.dtypes

song_clean       object
artist_clean     object
release_year    float64
combined         object
first             int64
year              int64
playcount         int64
fg                int64
dtype: object

In [197]:
rock['release_year'].apply(pd.to_numeric)
rock.dtypes

song_clean       object
artist_clean     object
release_year    float64
combined         object
first             int64
year              int64
playcount         int64
fg                int64
dtype: object

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [191]:
rock.min()

song_clean                     #9 Dream
artist_clean                .38 Special
release_year                          0
combined        #9 Dream by John Lennon
first                                 1
year                                  0
playcount                             0
fg                                    0
dtype: object

In [192]:
rock.max()

song_clean                     Ziggy Stardust
artist_clean                             a-ha
release_year                             2014
combined        Ziggy Stardust by David Bowie
first                                       1
year                                        1
playcount                                 142
fg                                        142
dtype: object

In [198]:
rock['release_year'].value_counts()

0.0       578
1973.0    104
1975.0     83
1977.0     83
1970.0     81
1971.0     75
1969.0     72
1980.0     70
1978.0     64
1979.0     63
1981.0     61
1967.0     61
1983.0     60
1976.0     56
1982.0     54
1984.0     51
1972.0     50
1974.0     48
1968.0     46
1985.0     39
1987.0     39
1986.0     37
1991.0     34
1989.0     32
1966.0     30
1988.0     29
1965.0     28
1994.0     25
1990.0     22
1993.0     19
1992.0     14
1964.0     14
1999.0     13
1995.0     10
1997.0      9
1963.0      9
1996.0      9
1998.0      6
2002.0      6
2005.0      5
2012.0      5
2004.0      5
2001.0      4
1962.0      3
2000.0      3
2007.0      3
2003.0      3
2011.0      3
2008.0      3
2013.0      2
2014.0      2
1961.0      1
2006.0      1
1958.0      1
1955.0      1
1071.0      1
Name: release_year, dtype: int64

# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [216]:
def track_detail(row):
    print('Song: ',row['song_clean'])
    print('Artist: ',row['artist_clean'])
    print('released before 1970: ',row['release_year'] < 1970)

In [214]:
rock.iloc[0]

song_clean                     Caught Up in You
artist_clean                        .38 Special
release_year                               1982
combined        Caught Up in You by .38 Special
first                                         1
year                                          1
playcount                                    82
fg                                           82
Name: 0, dtype: object

In [215]:
track_detail(rock.iloc[0])

('Song: ', 'Caught Up in You')
('Artist: ', '.38 Special')
('released before 1970: ', False)




## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [217]:
def convert_to_float(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

## Apply these functions to your dataset

In [219]:
rock.apply(convert_to_float)

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1,1,82,82
1,NaN,NaN,0.0,NaN,1,0,3,0
2,NaN,NaN,1981.0,NaN,1,1,85,85
3,NaN,NaN,1980.0,NaN,1,1,18,18
4,NaN,NaN,1975.0,NaN,1,1,1,1
5,NaN,NaN,2000.0,NaN,1,1,13,13
6,NaN,NaN,2000.0,NaN,1,1,1,1
7,NaN,NaN,2002.0,NaN,1,1,6,6
8,NaN,NaN,1992.0,NaN,1,1,3,3
9,NaN,NaN,1985.0,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [ ]:
rock.describe()



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



